In [ ]:
import requests
import datetime
import getpass

BOARD_TOKENS = {
    "dl":"xxxxxx",
    "house":"xxxxxx",
    "finance":"xxxxxx",
    "personal_tasks":"xxxxxx",
    "learning_dl":"xxxxxx",    
}

COLUMN_NAMES = {'todo':'To-do', 'today':"Due today", 'now':"In progress", "done":"Done"}
COLOR_OPTIONS = ['yellow', 'white', 'red', 'green', 'blue', 'purple', 'orange', 'cyan', 'brown', 'magenta']
TZ_OFFSET=6

class APIRequestMaker:
    def __init__(self, api_token):
        self.api_token = api_token
        
        self.allowed_methods = ['get','post','put','patch','delete']
        
        
        self.base_api = "https://kanbanflow.com/api/v1"
        
    def request(self, endpoint, method="get", payload=None):
        method = method.lower()
        assert method in self.allowed_methods
        req_method = getattr(requests,method)
        res = req_method(
            url=self.base_api.strip("/") + "/" + endpoint.strip("/"),
            json=payload,
            params={"apiToken":self.api_token}
        )
        return res
 
class Task:
    '''
    Object to hold data for a new task
    '''
    def __init__(self, name, board_name, column_name, color, description, subtask_dict_list = None, label_dict_list=None, due_date_ts=None, target_due_column_name=None):
        self.allowed_colors = COLOR_OPTIONS
        
        assert board_name in BOARD_TOKENS, "No token for this board"
        assert column_name in COLUMN_NAMES, "Invalid Column Name"
        self.column_name = COLUMN_NAMES[column_name]
        self.api_requester = APIRequestMaker(api_token=BOARD_TOKENS[board_name])
        
        self._board_info = self._get_board_info(board_name)
        
        self.name = name
        self.column_id = self._get_column_id(self.column_name)
        assert color.lower() in self.allowed_colors
        self.color = color.lower()
        self.description = description        
        self.subtask_dict_list = self._get_subtask_dict_list(subtask_dict_list) if subtask_dict_list else [] # self._get_subtask_dict_list(subtask_dict_list)  if subtask_dict_list else None    
        self.label_dict_list = self._get_label_dict_list(label_dict_list) if label_dict_list else []
        self.due_date = self._get_due_date(due_date_ts) if due_date_ts else []
        self.target_due_column_id = self._get_target_due_column_id(target_due_column_name) if target_due_column_name else []
        

    def _get_board_info(self, board_name):
        bi =  self.api_requester.request(endpoint="board", method="get", payload=None).json()
        print(bi)
        return bi
        
    def _get_due_date(self, due_date_ts):
        return datetime.datetime.strftime(
            due_date_ts + datetime.timedelta(hours=TZ_OFFSET),
            "%Y-%m-%dT%H:%M:%SZ",
        )
        
    def _get_column_id(self, column_name):
        print(column_name)
        print(self._board_info['columns'])
        return list(filter(lambda x: x['name'] == column_name, self._board_info['columns']))[0]['uniqueId']
    
    def _get_target_due_column_id(self, target_due_column_name):
        cn = COLUMN_NAMES[target_due_column_name]
        tcid =  str(list(filter(lambda x: x['name'] == cn, self._board_info['columns']))[0]['uniqueId'])
        print(tcid)
        return tcid
        
    def _get_subtask_dict_list(self, subtask_dict_list):
        new_st_d_l = []
        for st_d in subtask_dict_list:
            new_st_d = {'name':st_d['name']}
            if 'dt' in st_d:
                dt = datetime.datetime.strftime(
                    st_d[dt] + datetime.timedelta(hours=TZ_OFFSET),
                    "%Y-%m-%dT%H:%M:%SZ",
                )
                print(type(dt))
                new_st_d['dueDateTimestamp'] = dt
            new_st_d_l.append(new_st_d)
        return new_st_d_l
    
    def _make_task(self):
        rq_dict = {
            "name":self.name,
            "columnId":self.column_id,
            "color":self.color,
            "description":self.description,  
        }
        if self.label_dict_list:
            rq_dict['labels'] = self.label_dict_list
        if self.subtask_dict_list:
            rq_dict['subTasks'] = self.subtask_dict_list
        if self.due_date:
            rq_dict['dates'] = [{"dueTimestamp":self.due_date, "targetColumnId": str(self.target_due_column_id)}]
        
        r = self.api_requester.request(endpoint="tasks", method="post", payload=rq_dict)
        self.task_id = r.json().get("taskId")
        print(r.status_code)
        print(r.json())
        assert self.task_id
        
    def _make_labels(self):
        print(self.label_dict_list)
        r = self.api_requester.request(endpoint=f"tasks/{self.task_id}/labels", method="post", payload={
            "labels":self.label_dict_list
        })
        print(r.status_code)
        print(r.json())
        
        
    def _make_subtasks(self):
        raise NotImplementedError
        
    def _make_due_date(self):
        raise NotImplementedError
  
    def create(self):
        self._make_task()
        
    def _get_label_dict_list(self, label_dict_list):
        return label_dict_list
 

In [ ]:
print(f"Board: {BOARD_TOKENS.keys()}")
print(f"Cols: {COLUMN_NAMES.keys()}")
print(f"Colors: {COLOR_OPTIONS}")

# Edit this to make a task

In [ ]:
year = 2021
month = 2
day = 14
hour = 11
minute=13
second = 0
name = '''
stuff and things
'''

description = '''

'''

board = 'dl'

color='red'
col = 'todo'
target_col = 'done'

labels = [
    {"name":"Stuff","pinned":True}
]

subtasks = [
    {"name":"Do thing"}
]

due_date = datetime.datetime(year, month, day, hour, minute, second)

t = Task(
    name=name.strip(),
    board_name=board,
    column_name=col,
    color=color,
    description=description.strip(),
    subtask_dict_list = subtasks,
    label_dict_list=labels,
    due_date_ts=due_date,
    target_due_column_name=target_col,
)
t.create()
